In [1]:
#Import Dependencies
%matplotlib inline
import os
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import json
from pprint import pprint
from datetime import datetime
from datetime import timedelta 
import time
import matplotlib.pyplot as plt

In [2]:
#Variables
start_date = "2008-01-01"
initial_date = datetime(2008, 1, 1)
end_date = (datetime.today() - timedelta(days = 2)).date()
print(str(initial_date.date()))

2008-01-01


In [3]:
executable_path = {'executable_path': 'C:/Users/joaquin/OneDrive/Curso/Proy_final/chromedriver.exe'}
browser = Browser('chrome', **executable_path)

In [4]:
#Banxico URL
url = 'https://www.banxico.org.mx/publicaciones-y-prensa/anuncios-de-las-decisiones-de-politica-monetaria/anuncios-politica-monetaria-t.html'
browser.visit(url)
api_url = "https://www.banxico.org.mx/SieAPIRest/service/v1/series/"
banxico_key = "85c8edeab72128102cdad764f0a2d4532c336d22690a51104b500fb6f077ed4c"

In [5]:
html = browser.html
soup = bs(html, 'html.parser')

In [6]:
links = []
for link in soup.find_all('a'):
    if ('/publicaciones-y-prensa/anuncios-de-las-decisiones-de-politica-monetaria') in link.get('href'):
        partial_link = link.get('href')
        links.append('http://www.banxico.org.mx' + partial_link)
links = links[1:]
#links

In [7]:
dates = []
for date in soup.find_all("td"):
    data = date.get_text()
    dates.append(data)
del dates[1::2]

In [8]:
import re
dates2 = []
for index in range(len(dates)):
    dates2.append(re.sub("\s", "", dates[index])) 


In [9]:
dates3 = []
for index in range(len(dates)):
    dates3.append(re.sub("[\/]", "", dates2[index])) 
#print(dates3)

In [ ]:
name_counter = 0
for i in links:
    r = requests.get(i)
    out_file = open(f'{dates3[name_counter]}.pdf', 'wb')
    out_file.write(r.content)
    name_counter += 1

In [ ]:
#Guillermo 
#pdf to text

In [23]:
final_dates = []
for date in dates2:
    dates = datetime.strptime(date, "%d/%m/%y")
    if dates > initial_date:
        final_dates.append(datetime.strftime(dates, "%Y-%m-%d"))
final_dates.append(datetime.strftime(end_date, "%Y-%m-%d"))

In [11]:
#MXN Lvl
Series = "SF43788"

fxt = []
for fecha in final_dates:
    url_consulta = [f"{api_url}{Series}/datos/{fecha}/{fecha}?token={banxico_key}"]
    for query in url_consulta: 
        data = requests.get(query).json()
        fxt.append(data["bmx"]["series"][0]["datos"][0]["dato"])
time.sleep(120) #tiempo del API

In [12]:
Series = "SF43878"

tasa = []
for fecha in final_dates:
    try:
        url_consulta = [f"{api_url}{Series}/datos/{fecha}/{fecha}?token={banxico_key}"]
        for query in url_consulta: 
            data = requests.get(query).json()
            tasa.append(data["bmx"]["series"][0]["datos"][0]["dato"])
    except:
        tasa.append(0)
time.sleep(120) #tiempo del API   
    
        

In [27]:
Series = "SP30578"

#Total Data
total_query_url = f"{api_url}{Series}/datos/{start_date}/{end_date}?token={banxico_key}"
total_data = requests.get(total_query_url).json()
#print(total_data)
data_total = total_data['bmx']['series'][0]['datos']
data_total

[{'fecha': '01/01/2008', 'dato': '3.70000000'},
 {'fecha': '01/02/2008', 'dato': '3.72000000'},
 {'fecha': '01/03/2008', 'dato': '4.25000000'},
 {'fecha': '01/04/2008', 'dato': '4.55000000'},
 {'fecha': '01/05/2008', 'dato': '4.95000000'},
 {'fecha': '01/06/2008', 'dato': '5.26000000'},
 {'fecha': '01/07/2008', 'dato': '5.39000000'},
 {'fecha': '01/08/2008', 'dato': '5.57000000'},
 {'fecha': '01/09/2008', 'dato': '5.47000000'},
 {'fecha': '01/10/2008', 'dato': '5.78000000'},
 {'fecha': '01/11/2008', 'dato': '6.23000000'},
 {'fecha': '01/12/2008', 'dato': '6.53000000'},
 {'fecha': '01/01/2009', 'dato': '6.28000000'},
 {'fecha': '01/02/2009', 'dato': '6.20000000'},
 {'fecha': '01/03/2009', 'dato': '6.04000000'},
 {'fecha': '01/04/2009', 'dato': '6.17000000'},
 {'fecha': '01/05/2009', 'dato': '5.98000000'},
 {'fecha': '01/06/2009', 'dato': '5.74000000'},
 {'fecha': '01/07/2009', 'dato': '5.44000000'},
 {'fecha': '01/08/2009', 'dato': '5.08000000'},
 {'fecha': '01/09/2009', 'dato': '4.8900

In [28]:
#Match fechas
data_total_df = pd.DataFrame(data_total)
data_total_df['fecha'] = pd.to_datetime(data_total_df['fecha'], format='%d/%m/%Y')
data_total_df['key'] = pd.to_datetime(data_total_df['fecha']).dt.to_period('M')

,dato,fecha,key
0,3.70000000,2008-01-01,2008-01
1,3.72000000,2008-02-01,2008-02
2,4.25000000,2008-03-01,2008-03
3,4.55000000,2008-04-01,2008-04
4,4.95000000,2008-05-01,2008-05
5,5.26000000,2008-06-01,2008-06
6,5.39000000,2008-07-01,2008-07
7,5.57000000,2008-08-01,2008-08
8,5.47000000,2008-09-01,2008-09
9,5.78000000,2008-10-01,2008-10


In [24]:

Banxico_dict = {"MXN": fxt,  "Tasa": tasa }
                
Banxico_data = pd.DataFrame(Banxico_dict)
Banxico_data["Fecha"] = final_dates

Banxico_data = Banxico_data[['Fecha', 'MXN', "Tasa"]]
Banxico_data = Banxico_data.sort_values('Fecha')
Banxico_data = Banxico_data[Banxico_data.Tasa != 0]
Banxico_data = Banxico_data.reset_index(drop = True)

In [ ]:
#Xime Word count feed a este DF



In [25]:

Banxico_data['Inflacion'] = 0
#Banxico_data['Crecimiento'] = 0
#Banxico_data['Desempleo'] = 0
Banxico_data['Dif'] = 0
Banxico_data['Tasa'] = Banxico_data['Tasa'].astype(float)
Banxico_data

,Fecha,MXN,Tasa,Inflacion,Dif
0,2009-11-27,12.9370,5.0650,0,0
1,2010-01-15,12.7040,5.0825,0,0
2,2010-02-19,12.8085,5.0750,0,0
3,2010-03-19,12.5685,5.0300,0,0
4,2010-04-16,12.2810,5.0200,0,0
5,2010-05-21,13.0130,5.0265,0,0
6,2010-06-18,12.5270,4.9850,0,0
7,2010-07-16,12.9310,5.0100,0,0
8,2010-08-20,12.7850,5.0200,0,0
9,2010-09-24,12.5500,4.9600,0,0


In [ ]:
for i in range(1, len(Banxico_data) -1 ):
    Banxico_data.loc[i, 'Dif'] = 25 * round((100 * (Banxico_data.loc[i, 'Tasa'] - Banxico_data.loc[i+1, 'Tasa']))/25)

In [ ]:
Banxico_data

In [ ]:
# Assign X (data) and y (target)
X = Banxico_data.drop(["Dif", "Fecha"], axis=1)
y = Banxico_data["Dif"]
print(X.shape, y.shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

In [ ]:
 classifier.fit(X_train, y_train)

In [ ]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

In [ ]:
predictions = classifier.predict(X_test)
print(f"First 10 Predictions:   {predictions[:50]}")
print(f"First 10 Actual labels: {y_test[:50].tolist()}")

In [ ]:
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)